In [3]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from sklearn.metrics import (accuracy_score, balanced_accuracy_score, precision_score, 
                             recall_score, f1_score, roc_auc_score, matthews_corrcoef)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Load the saved models and data
with open('churn_models_and_data.pkl', 'rb') as f:
    saved_data = pickle.load(f)

models = saved_data['models']
X_train = saved_data['X_train']
X_test = saved_data['X_test']
y_train = saved_data['y_train']
y_test = saved_data['y_test']
feature_names = saved_data['feature_names']

y_test = pd.Series(y_test, dtype='int')  # Ensure y_test is a Series for consistency
y_train = pd.Series(y_train, dtype='int')  # Ensure y_train is a Series for consistency

# Load existing evaluation results
eval_results = pd.read_csv('model_evaluation_results.csv')

# Reduced hyperparameter grids for memory efficiency
param_grids = {
    'Logistic Regression': {
        'C': [0.1, 1, 10],  # Reduced from 6 to 3
        'penalty': ['l2'],   # Removed 'l1' to simplify
        'solver': ['liblinear'],  # Removed 'saga'
        'max_iter': [100]    # Reduced from 3 to 1
    },
    'Random Forest': {
        'n_estimators': [50, 100],  # Reduced from 3 to 2
        'max_depth': [10, 20],     # Reduced from 4 to 2
        'min_samples_split': [2, 5],  # Reduced from 3 to 2
        'min_samples_leaf': [1, 2],   # Reduced from 3 to 2
        'bootstrap': [True]         # Removed False option
    },
    'Gradient Boosting': {
        'n_estimators': [50, 100],  # Reduced from 3 to 2
        'learning_rate': [0.1, 0.2], # Reduced from 3 to 2
        'max_depth': [3, 5],        # Reduced from 3 to 2
        'min_samples_split': [2],   # Reduced from 2 to 1
        'min_samples_leaf': [1]     # Reduced from 2 to 1
    }
}

# Memory-efficient evaluation function
def evaluate_model(model, X, y):
    try:
        y_pred = model.predict(X)
        y_prob = model.predict_proba(X)[:, 1] if hasattr(model, "predict_proba") else None
        
        metrics = {
            'Accuracy': accuracy_score(y, y_pred),
            'Balanced Accuracy': balanced_accuracy_score(y, y_pred),
            'Precision': precision_score(y, y_pred),
            'Recall': recall_score(y, y_pred),
            'F1 Score': f1_score(y, y_pred),
            'MCC': matthews_corrcoef(y, y_pred)
        }
        
        if y_prob is not None:
            metrics['AUC'] = roc_auc_score(y, y_prob)
        else:
            metrics['AUC'] = np.nan
            
        return metrics
    except Exception as e:
        print(f"Error evaluating {model.__class__.__name__}: {str(e)}")
        return {k: np.nan for k in ['Accuracy', 'Balanced Accuracy', 'Precision', 
                                   'Recall', 'F1 Score', 'AUC', 'MCC']}

# Perform memory-efficient tuning
tuned_models = {}
results = []

for model_name in models.keys():
    print(f"\n=== Tuning {model_name} (memory-efficient version) ===")
    
    model = models[model_name]
    param_grid = param_grids[model_name]
    
    # Use RandomizedSearchCV instead of GridSearchCV for efficiency
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_grid,
        n_iter=5,  # Reduced number of iterations
        cv=3,      # Reduced from 5 to 3 folds
        scoring='roc_auc',
        n_jobs=1,  # Run sequentially to reduce memory usage
        verbose=1,
        random_state=42
    )
    
    # Fit with memory management
    try:
        random_search.fit(X_train, y_train)
    except MemoryError:
        print(f"Memory error with {model_name}. Skipping to next model.")
        continue
    
    best_model = random_search.best_estimator_
    tuned_models[model_name] = best_model
    
    # Evaluate with reduced memory footprint
    train_metrics = evaluate_model(best_model, X_train, y_train)
    test_metrics = evaluate_model(best_model, X_test, y_test)
    
    results.append({
        'Model': model_name,
        'Best Params': str(random_search.best_params_),
        **{f'Train {k}': v for k, v in train_metrics.items()},
        **{f'Test {k}': v for k, v in test_metrics.items()}
    })

    # Clear memory between models
    del random_search
    import gc
    gc.collect()

# Save results incrementally to avoid memory issues
tuning_results = pd.DataFrame(results)
try:
    tuning_results.to_csv('hyperparameter_tuning_results_light.csv', index=False)
    
    # Simple comparison (avoids loading both datasets in memory)
    comparison = tuning_results[['Model', 'Test Accuracy', 'Test Balanced Accuracy', 
                               'Test Precision', 'Test Recall', 'Test F1 Score', 
                               'Test AUC', 'Test MCC']].copy()
    comparison.columns = [col.replace('Test ', '') for col in comparison.columns]
    
    # Save tuned models one by one
    with open('tuned_churn_models_light.pkl', 'wb') as f:
        pickle.dump({'tuned_models': tuned_models}, f)
        
    # Lightweight visualization
    plt.figure(figsize=(12, 6))
    metrics_to_plot = ['Accuracy', 'F1 Score', 'AUC']
    
    for i, metric in enumerate(metrics_to_plot, 1):
        plt.subplot(1, 3, i)
        sns.barplot(x='Model', y=metric, data=comparison)
        plt.title(metric)
        plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig('model_comparison_plots_light.png')
    plt.close()  # Close figure to free memory
    
    print("\n=== Lightweight Tuning Complete ===")
    print("Saved files:")
    print("- tuned_churn_models_light.pkl: Contains the tuned models")
    print("- hyperparameter_tuning_results_light.csv: Tuning results")
    print("- model_comparison_plots_light.png: Basic visualization")
    
except MemoryError:
    print("Memory error during saving results. Results not saved.")


=== Tuning Logistic Regression (memory-efficient version) ===
Fitting 3 folds for each of 3 candidates, totalling 9 fits

=== Tuning Random Forest (memory-efficient version) ===
Fitting 3 folds for each of 5 candidates, totalling 15 fits

=== Tuning Gradient Boosting (memory-efficient version) ===
Fitting 3 folds for each of 5 candidates, totalling 15 fits

=== Lightweight Tuning Complete ===
Saved files:
- tuned_churn_models_light.pkl: Contains the tuned models
- hyperparameter_tuning_results_light.csv: Tuning results
- model_comparison_plots_light.png: Basic visualization
